# 3DMedSAM-FDA Experiments

이 노트북은 3DMedSAM-FDA 모델의 학습, 평가 및 시각화 파이프라인을 실행하는 명령어들을 설명합니다.

### 목차 

0. [Setting](#0-세팅)
1. [Training](#1-모델-학습-training): `train.py`, `train_resume.py`, `train_fftlocal_A100.py` 실행 
2. [Testing (Metrics)](#2-testing-metrics): `test.py` 실행 
3. [Inference (Single Case)](#3-inference-single-case): `test_sample.py` 실행
4. [Model Analysis](#4-model-analysis): `count_params.py` 실행
5. [Visualization (2D Slices)](#5-visualization-2d-slices): `test_visualize.py, test_visualize_ptz.py` 실행
6. [Visualization (3D)](#6-visualization-3d): `visualize_3d.py` 실행

### 0. 세팅

Global Settings 먼저 공통으로 사용할 경로와 데이터셋을 설정합니다.

In [ ]:
import os

# 사용자 설정 변수
DATASET = "colon"  # choices: ["kits", "pancreas", "lits", "colon"]
SNAPSHOT_PATH = "/workspace/weights/"
DATA_ROOT = f"/workspace/dataset/Task10_Colon/"  # 데이터셋 경로에 맞게 수정
GPUS = "0"

# 환경 변수 설정 (필요시)
os.environ["CUDA_VISIBLE_DEVICES"] = GPUS

### 1. 모델 학습 (Training)

모델 학습을 수행합니다. 상황에 따라 적절한 스크립트를 선택하여 실행하세요.

##### 1.1 기본 학습 (`train.py`)
가장 기본적인 학습 루프를 실행합니다.

In [ ]:
#
!python -u train.py \
    --data {DATASET} \
    --snapshot_path "{SNAPSHOT_PATH}" \
    --data_prefix "{DATA_ROOT}" \
    --max_epoch 200 \
    --batch_size 1 \
    --num_worker 6 \
    --lr 4e-4

##### 1.2 학습 재개 (`train_resume.py`)
중단된 시점(last.pth.tar)부터 학습을 이어갈 때 사용합니다. --resume 옵션이 포함되어 있습니다.

In [ ]:
#
!python -u train_resume.py \
    --data {DATASET} \
    --snapshot_path "{SNAPSHOT_PATH}" \
    --data_prefix "{DATA_ROOT}" \
    --max_epoch 300 \
    --resume

##### 1.3 A100과 같은 불안정한 환경에서의 FFT-based DPA 모델 학습 (`train_fftlocal_A100.py`)
빈 마스크(Empty Mask)를 스킵하거나 NaN/Inf 값을 체크하는 등의 안정화 로직이 추가된 버전입니다.

In [ ]:
#
!python -u train_fftlocal_A100.py \
    --data {DATASET} \
    --snapshot_path "{SNAPSHOT_PATH}" \
    --data_prefix "{DATA_ROOT}" \
    --max_epoch 200

### 2. Testing (Metrics)

학습된 모델의 정량적 성능(Dice Score, NSD)을 평가합니다.

##### 2.1 전체 테스트 셋 평가 (`test.py`)

In [ ]:
#
# --checkpoint: 'best' (최고 성능) 또는 'last' (마지막 에폭)
# --num_prompts: 추론 시 사용할 포인트 프롬프트 개수
!python -u test.py \
    --data {DATASET} \
    --snapshot_path "{SNAPSHOT_PATH}" \
    --data_prefix "{DATA_ROOT}" \
    --num_prompts 1 \
    --checkpoint best

### 3. Inference (Single Case)

특정 케이스 하나에 대해 실제 예측 결과물(NIfTI 파일)을 생성합니다.

##### 3.1 단일 케이스 추론 (`test_sample.py`)
이미지, 정답 마스크, 예측 마스크, 포인트 위치를 각각 `.nii.gz` 파일로 저장합니다.

In [ ]:
#
CASE_ID = "001"
IMG_PATH = os.path.join(DATA_ROOT, f"imagesTr/{DATASET}_{CASE_ID}.nii.gz")
MASK_PATH = os.path.join(DATA_ROOT, f"labelsTr/{DATASET}_{CASE_ID}.nii.gz")
OUTPUT_DIR = f"/workspace/samples/{DATASET}"

!python test_sample.py \
    --img_path "{IMG_PATH}" \
    --mask_path "{MASK_PATH}" \
    --snapshot_path "{SNAPSHOT_PATH}" \
    --data {DATASET} \
    --case_id {CASE_ID} \
    --output_dir "{OUTPUT_DIR}" \
    --num_prompts 1 \
    --checkpoint best

### 4. Model Analysis

모델의 파라미터 수를 분석합니다.

##### 4.1 파라미터 수 확인 (`count_params.py`)
전체 파라미터 대비 학습 가능한(Tuned) 파라미터의 비율을 확인합니다.

In [ ]:
#
!python count_params.py

### 5. Visualization (2D Slices)

테스트 셋에 대한 예측 결과를 2D 슬라이스 이미지로 저장하여 시각적으로 확인합니다.

##### 5.1 랜덤 프롬프트 시각화 (`test_visualize.py`)
종양 내 랜덤 위치에 프롬프트를 생성하고 결과를 시각화합니다.

In [ ]:
#
# 결과는 snapshot_path/test_YYYYMMDD-HHMMSS 폴더에 저장됩니다.
!python -u test_visualize.py \
    --data {DATASET} \
    --snapshot_path "{SNAPSHOT_PATH}" \
    --data_prefix "{DATA_ROOT}" \
    --num_prompts 1

##### 5.2 특정 Z 슬라이스 지정 시각화 (`test_visualize_ptz.py`)
특정 깊이(Z-index)에 강제로 프롬프트를 찍어 해당 슬라이스의 예측 성능을 집중적으로 확인합니다.

In [ ]:
#
TARGET_Z = 100  # 보고 싶은 슬라이스 번호

!python -u test_visualize_ptz.py \
    --data {DATASET} \
    --snapshot_path "{SNAPSHOT_PATH}" \
    --data_prefix "{DATA_ROOT}" \
    --num_prompts 1 \
    --pt_z {TARGET_Z}

### 6. Visualization (3D)

모델 추론 결과를 3D 렌더링하여 시각적으로 확인합니다.

##### 6.1 3D 렌더링 및 출력 (`visualize_3d.py`)


실행 전 설정 사항:
- `$pip install pyvista nibabel numpy` 설치 
- `$apt-get update`, `$apt-get install -y libgl1-mesa-glx xvfb` 설치
- `visualize_3d.py` 파일 내 `img_file`, `mask_file` 경로 설정

In [ ]:
#
!python visualize_3d.py

##### 6.2
또는 `test_sample.py`에서 생성된 NIfTI 파일을 사용하여 3D 렌더링을 수행하고 노트북에서 바로 확인합니다.

In [ ]:
#
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from visualize_3d import save_3d_visualization

# 시각화할 파일 경로 (Inference 단계에서 생성된 파일)
# 파일명 형식은 test_sample.py의 출력에 따름
viz_img_path = os.path.join(OUTPUT_DIR, f"imaging_{CASE_ID}.nii.gz")
viz_mask_path = os.path.join(OUTPUT_DIR, f"prediction_{CASE_ID}.nii.gz") # 또는 mask_{CASE_ID}.nii.gz (GT)
output_png = "result_3d.png"

# 1. 3D 렌더링 실행 및 이미지 저장
save_3d_visualization(viz_img_path, viz_mask_path, output_file=output_png)

# 2. 노트북에 이미지 출력
if os.path.exists(output_png):
    img = mpimg.imread(output_png)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Case {CASE_ID} 3D Visualization")
    plt.show()
else:
    print("이미지 생성 실패")